In [18]:
import torch as t
import matplotlib.pyplot as plt
import random
from filter import Filter

In [19]:
# Params to change
dt = 0.1
sigma = 2
x_0 = t.tensor([1, 1, 0, 0], dtype=t.float64)
mu = t.tensor([0, 0, 0, 0], dtype=t.float64)
mu_y = t.tensor([0, 0], dtype=t.float64)
steps = 150
k_shots = 25
min_max = 100
seed = 987

random.seed(seed)
t.manual_seed(987)

In [20]:
filter = Filter(dim_x=4, dim_y=2)

In [21]:
F = t.tensor([[1, 0, dt,  0],
              [0, 1,  0, dt],
              [0, 0,  1,  0],
              [0, 0,  0,  1]], dtype=t.float64)
filter.F = F

In [22]:
filter.sigma = sigma

In [23]:
theta = t.tensor([t.pi/3], dtype=t.float64)
accel_north = t.sin(theta)
accel_east = t.cos(theta)
B = t.tensor([0, 0, accel_north*dt,  accel_east*dt], dtype=t.float64)
filter.B = B

In [24]:
H = t.tensor([[1, 0, 0, 0],
              [0, 1, 0, 0]], dtype=t.float64)
filter.H = H

In [25]:
q = 0.1 
diag_Q = t.tensor([q, q, q, q], dtype=t.float64)
Q = t.diag(diag_Q)
filter.Q = Q

In [26]:
r = 0.1
diag_R = t.tensor([r, r], dtype=t.float64)
R = t.diag(diag_R)
filter.R = R

In [27]:
diag_P_0 = t.tensor([4, 4, 3, 3], dtype=t.float64)
P_0 = t.diag(diag_P_0)
filter.P_0 = P_0

In [28]:
def generate_x_state_clean(F, x_0, B, steps):
    n = x_0.shape[0]
    x_state = t.zeros((n, steps), dtype=t.float64)
    x_state[:, 0] = x_0
    u = 1
    
    for k in range (1, steps):
        x_state[:, k] = F @ x_state[:, k-1] + B*u

    return x_state

In [29]:
def generate_y_observation_clean(H, x):
    m = H.shape[0]
    steps = x.shape[1]
    y_obs = t.zeros((m, steps), dtype=t.float64)

    for k in range (1, steps):
        y_obs[:, k] = H @ x[:, k - 1]
    
    return y_obs

In [30]:
def generate_shot_noise(vec, k_shots, min_max):
    num_of_params = vec.shape[0]
    steps = vec.shape[1]
    vec_shots = t.zeros(vec.shape, dtype=t.float64)

    min = -min_max
    max = min_max

    vec_shot_ind = random.sample(range(0, steps), k_shots)
    vec_shot_val = min + (max - min) * t.rand(k_shots, dtype=t.float64)
    
    for id, k in enumerate(vec_shot_ind):
        for i in range(num_of_params):
            vec_shots[i, k] += vec_shot_val[id]

    return vec_shots


In [31]:
def generate_normal_noise(steps, mu, COV):
    mvn = t.distributions.MultivariateNormal(mu, covariance_matrix=COV)
    N_mu_Q = mvn.sample((steps,)).T
    return N_mu_Q

In [32]:
COV = t.diag(t.tensor([0.1, 0.1], dtype=t.float64))
x = generate_x_state_clean(F, x_0, B, steps)
y = generate_y_observation_clean(H, x)
y += generate_normal_noise(steps, mu_y, R) 
y += generate_shot_noise(y, k_shots, min_max)

In [33]:
filtered_KF = filter(y, "KF")
filtered_MCF = filter(y, "MCF")
filtered_MCCKF = filter(y, "MCCKF")

In [34]:
# %matplotlib inline
%matplotlib qt
time = t.arange(0, steps*dt, dt)

plt.figure(figsize=(10, 5))  # Размер графика
plt.plot(time.numpy(), x[0, :].numpy(), color='black')
plt.plot(time.numpy(), y[0, :].numpy(), color='red')
plt.plot(time.numpy(), filtered_KF[0, :].numpy(), color='green')
plt.plot(time.numpy(), filtered_MCF[0, :].numpy(), color='orange')
plt.plot(time.numpy(), filtered_MCCKF[0, :].numpy(), color='blue')